In [119]:
import requests
import base64
import datetime
from urllib.parse import urlencode

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import time
import re
import json

import datetime

In [120]:
dfGenres = pd.read_csv('dfGenres.csv')
totalHist = pd.read_csv('totalHist.csv')

In [189]:
dfGenres

,artistName,genres
0,Imagine Dragons,"['modern rock', 'rock']"
1,iLOVEFRiDAY,['viral pop']
2,Panic! At The Disco,"['baroque pop', 'emo', 'modern rock', 'pop punk']"
3,Oliver Tree,['alternative hip hop']
4,88rising,"['asian american hip hop', 'pop']"
...,...,...
1536,Anuel AA,"['latin', 'reggaeton flow', 'trap latino']"
1537,Enrique Iglesias,"['dance pop', 'latin', 'latin pop', 'mexican p..."
1538,Myke Towers,['trap latino']
1539,Pedro CapÃ³,"['latin', 'latin pop', 'puerto rican pop']"


In [193]:
def convToList(row):
    if pd.isna(row['genres']):
        return['No Genre']
    else:
        return row['genres'][1:-1].replace('\'', '').split(', ')


In [196]:
dfGenres['genres'] = dfGenres.apply(convToList, axis=1)

In [205]:
genreList = []
for genres in dfGenres['genres']:
    for genre in genres:
        genreList.append(genre)

In [210]:
uniqueGenres = pd.Series(genreList).unique()

In [221]:
for genre in uniqueGenres:
    dfGenres[genre] = 0
for genre in uniqueGenres:
    totalHist[genre] = 0

In [239]:
def fillOutGenres(row):    
    for genre in row['genres']:
        row[genre] = 1
    return row
    

In [241]:
dfGenres = dfGenres.apply(fillOutGenres, axis=1)

In [242]:
dfGenres

,artistName,genres,genre,modern rock,rock,viral pop,baroque pop,emo,pop punk,alternative hip hop,...,polynesian pop,experimental,experimental rock,popgaze,noise punk,deconstructed club,modern salsa,salsa,reggaeton flow,puerto rican pop
0,Imagine Dragons,"[modern rock, rock]",NaN,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,iLOVEFRiDAY,[viral pop],NaN,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Panic! At The Disco,"[baroque pop, emo, modern rock, pop punk]",NaN,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Oliver Tree,[alternative hip hop],NaN,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,88rising,"[asian american hip hop, pop]",NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,Anuel AA,"[latin, reggaeton flow, trap latino]",NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1537,Enrique Iglesias,"[dance pop, latin, latin pop, mexican pop, pop...",NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1538,Myke Towers,[trap latino],NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1539,Pedro CapÃ³,"[latin, latin pop, puerto rican pop]",NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [243]:
totalHist

,endTime,artistName,trackName,msPlayed,modern rock,rock,viral pop,baroque pop,emo,pop punk,...,polynesian pop,experimental,experimental rock,popgaze,noise punk,deconstructed club,modern salsa,salsa,reggaeton flow,puerto rican pop
0,2018-11-14 00:01,Imagine Dragons,Stuck,190853,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2018-11-14 00:04,Imagine Dragons,Love,166360,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2018-11-14 00:08,Imagine Dragons,Birds,219440,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2018-11-14 00:12,Imagine Dragons,Burn Out,273880,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2018-11-14 00:16,Imagine Dragons,Real Life,247546,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30822,2019-11-14 23:32,My Chemical Romance,Welcome to the Black Parade,311106,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30823,2019-11-14 23:34,Tony Molina,Jasper's Theme,121509,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30824,2019-11-14 23:38,Imagine Dragons,Cool Out,217866,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30825,2019-11-14 23:41,5 Seconds of Summer,If Walls Could Talk,182292,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
